
Web Scraping Wikipedia using BeautifulSoup and Python

« Une théorie veut que n'importe quel article sur Wikipedia pointe au final sur la philosophie. Pour la vérifier, il suffit de cliquer sur le premier lien d'un article Wikipedia qui mène à un autre article et ainsi de suite : à force, on tombe immanquablement sur l'article dédié à la philosophie. »

Ecrivons un programme qui vérifie cette théorie. Ce programme doit renvoyer la "distance" (int) qui sépare un article donné de l'article Philosophie.

In [10]:
import time
import urllib
import bs4
import requests

In [13]:
start_url = "https://fr.wikipedia.org/wiki/Math%C3%A9matiques"
end_url = "https://fr.wikipedia.org/wiki/Philosophie"    

In [21]:
def find_first_link(url):
    response = requests.get(url) 
    html = response.text
    soup = bs4.BeautifulSoup(html, "html.parser")

    content_div = soup.find(id="mw-content-text").find(class_="mw-parser-output")


    next_link = None # on initialise le lien suivant à none

    # Find all the direct children of content_div that are paragraphs
    for element in content_div.find_all("p", recursive=False):
        
       # on parcourt les éléments de la page et on cherche les liens les uns après les autres
        
        if element.find("a", recursive=False):
            next_link = element.find("a", recursive=False).get('href')
            break

    if not next_link:
        return

    # Build a full url from the relative next_link url
    first_link = urllib.parse.urljoin(
        'https://fr.wikipedia.org/', next_link)

    return first_link


def continue_crawl(search_history, target_url, max_steps=30):
    # Verifier si le lien c'est un lien vers phylosophie
    if search_history[-1] == target_url:
        print("le lien est trouvé")
        return False
        # on pose une limite à la recherche
    elif len(search_history) > max_steps:
        print("La recherche a pris une longueur non raisonable superieur à 30, nous quittons la recherche!")
        return False
    elif search_history[-1] in search_history[:-1]:
        print("Nous avons déjà trouver cette article nous quittons la recherche")
        return False
    else:
        return True


article_chain = [start_url]

while continue_crawl(article_chain, target_url):
    print(article_chain[-1])

    first_link = find_first_link(article_chain[-1])
    if not first_link:
        print("We've arrived at an article with no links, aborting search!")
        break

    article_chain.append(first_link)

    time.sleep(2)  # Slow things down so as to not hammer Wikipedia's servers

    
print('La distance parcourue entre ces 2 articles est : ' + str(len(article_chain)-1))



https://fr.wikipedia.org/wiki/Math%C3%A9matiques
https://fr.wikipedia.org/wiki/Connaissance
https://fr.wikipedia.org/wiki/Notion
https://fr.wikipedia.org/wiki/Connaissance_(philosophie)
le lien est trouvé
La distance parcourue entre ces 2 articles est : 4
